In [ ]:
import numpy as np
import scipy as sp
import scipy.sparse, scipy.io, scipy.optimize
from scipy.special import expit

import matplotlib.pyplot as plt
import matplotlib as mpl
import time
%matplotlib inline

CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a', # Color blind color cycle
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=CB_color_cycle) 

clip_01 = lambda M : np.clip(M, a_min=0, a_max=1)

Cells to load the networks. We use Cora as an example here. Note that preprocessing is not necessary for Cora.

In [ ]:
adj = sp.io.loadmat('cora.mat')['network']

# destroy diagonal and binarize
# adj.setdiag(0)
# adj.data = 1. * (adj.data > 0)

The LPCA loss function:

In [ ]:
def lpca_loss(factors, adj_s, rank): # adj_s = shifted adj with -1's and +1's
    n_row, n_col = adj_s.shape
    U = factors[:n_row*rank].reshape(n_row, rank)
    V = factors[n_row*rank:].reshape(rank, n_col)
    logits = U @ V
    prob_wrong = expit(-logits * adj_s)
    loss = (np.logaddexp(0,-logits*adj_s)).sum()# / n_element    
    U_grad = -(prob_wrong * adj_s) @ V.T# / n_element
    V_grad = -U.T @ (prob_wrong * adj_s)# / n_element
    print(loss)
    return loss, np.concatenate((U_grad.flatten(), V_grad.flatten()))

Initialize the embeddings (factors) and set a callback function; then, optimize:

In [ ]:
rank = 128
save_interval = 10 # the number of iterations after which to save the embeddings
factor_file_path = 'cora_embed.mat'

n_row, n_col = adj.shape
factors = -1+2*np.random.random(size=(np.sum(adj.shape)*rank)) # initalize uniformly on [-1,+1]
iter_num = 0
def callback_recm(x_i): # prints the loss and periodically saves the factors
    global iter_num
    iter_num += 1
    if iter_num % save_interval == 0:
        global factors, n_row, n_col, rank
        factors = x_i
        U = factors[:n_row*rank].reshape(n_row, rank)
        V = factors[n_row*rank:].reshape(rank, n_col)
        frob_error_norm = np.linalg.norm(clip_01(U@V) - adj) / sp.sparse.linalg.norm(adj)
        print(iter_num, "Frob_error_norm: ", frob_error_norm)
#         data = {'U':U, 'V':V}
#         scipy.io.savemat(factor_file_path, data)

U = factors[:n_row*rank].reshape(n_row, rank)
V = factors[n_row*rank:].reshape(rank, n_col)
frob_error_norm = np.linalg.norm(1.*(U @ V > 0) - adj) / sp.sparse.linalg.norm(adj)
print(frob_error_norm)

In [ ]:
res = scipy.optimize.minimize(lpca_loss, x0=factors, 
                              args=(-1 + 2*np.array(adj.todense()), rank), jac=True, method='L-BFGS-B', 
                              callback=callback_recm, 
                              options={'maxiter':2000}
                             )
factors = res.x
U = res.x[:n_row*rank].reshape(n_row, rank)
V = res.x[n_row*rank:].reshape(rank, n_col)
frob_error_norm = np.linalg.norm(clip_01(U@V) - adj) / sp.sparse.linalg.norm(adj)
print("Frob norm error: ", frob_error_norm)
# data = {'U':U, 'V':V}
# scipy.io.savemat(factor_file_path, data)

Cells for comparison to TSVD:

In [ ]:
# generate a rank k TSVD factorization of a small sparse matrix adj
def factor_TSVD(adj, k):
    w, v = np.linalg.eigh(np.array(adj.todense()))
    order = np.argsort(np.abs(w))[::-1]
    w = w[order[:k]]
    v = v[:,order[:k]]
    U_tsvd, V_tsvd = v * np.sqrt(np.abs(w))[None,:], (np.sign(w)*np.sqrt(np.abs(w)))[:,None] * v.T
    return U_tsvd, V_tsvd
U_tsvd, V_tsvd = factor_TSVD(adj, rank)

In [ ]:
# print the Frobenius errors of the LPCA and TSVD reconstructions, respectively 
print( np.linalg.norm(adj.todense() - expit(U@V)) / sp.sparse.linalg.norm(adj) )
print( np.linalg.norm(adj.todense() - clip_01(U_tsvd@V_tsvd)) / sp.sparse.linalg.norm(adj) )

Some code for other experiments:
A cell to create the triangle-dense toy graph from the paper, then a cell to plot reconstructions given embeddings:

In [ ]:
# Create the toy graph with n_tri triangles
def tri_path(n_tri, cycle=True, self_loops=True):
    tri_block = sp.sparse.coo_matrix([[0,1,1],[1,0,1],[1,1,0]])
    mat = sp.sparse.block_diag((tri_block,)*n_tri)
    diag = np.tile([0,0,1], n_tri)[:-1]
    mat += sp.sparse.diags([diag, diag], [-1,1])
    if cycle:
        mat[0,-1] = 1
        mat[-1,0] = 1
    if self_loops:
        mat += sp.sparse.identity(n_tri*3)
    return mat
adj = tri_path(100, cycle=True, self_loops=True)
plt.matshow(adj.todense()[:12,:][:,:12])
plt.matshow(adj.todense())

In [ ]:
fig, axs = plt.subplots(2,3)
fig.set_size_inches(6.5,3)
fig.set_dpi(300)

im = axs[0,0].imshow(adj[:12,:][:,:12].todense(), vmin=0,vmax=1)
axs[0,0].set_axis_off()
axs[1,0].imshow(adj.todense(), vmin=0,vmax=1)
axs[1,0].set_axis_off()
axs[0,0].set_title('True')

axs[0,1].imshow(expit(U@V)[:12,:][:,:12], vmin=0,vmax=1)
axs[0,1].set_axis_off()
axs[1,1].imshow(expit(U@V), vmin=0,vmax=1)
axs[1,1].set_axis_off()
axs[0,1].set_title('LPCA ' + str(rank))

axs[0,2].imshow(np.clip(U_tsvd@V_tsvd, a_min=0, a_max=1)[:12,:][:,:12], vmin=0,vmax=1)
axs[0,2].set_axis_off()
axs[1,2].imshow(np.clip(U_tsvd@V_tsvd, a_min=0, a_max=1), vmin=0,vmax=1)
axs[1,2].set_axis_off()
axs[0,2].set_title('TSVD ' + str(rank))

fig.colorbar(im, ax=axs.ravel().tolist(), ticks=[0, 0.5, 1], shrink=1.0)

Cells to calculate triangles in induced subgraphs and plot:

In [ ]:
from multiprocessing import Pool

n = adj.shape[0]
skip_int = 100 # collect data for every 10th point in the plot

adj_recon_lpca = expit(U@V)
deg_recon_lpca = np.array(adj_recon_lpca.sum(axis=1)).flatten()
deg_order_lpca = np.argsort(deg_recon_lpca)
deg_seq_recon_lpca = deg_recon[deg_order[::-skip_int]]

adj_recon_sort = np.copy(adj_recon_lpca[np.ix_(deg_order_lpca, deg_order_lpca)])

def get_num_tri(i):
    global adj_recon_sort
    sub_adj = adj_recon_sort[:i+1,:i+1]
    num_tri = (1. * sub_adj @ sub_adj @ sub_adj).diagonal().sum() / 6
    return num_tri

pool = Pool(5)
tri_seq_recon_lpca = np.array(pool.map(get_num_tri, np.arange(n)[::-skip_int]))
pool.close()
pool.join()

In [ ]:
adj_recon_tsvd = np.clip(U_tsvd@V_tsvd, a_min=0, a_max=1)
deg_recon_tsvd = np.array(adj_recon_tsvd.sum(axis=1)).flatten()
deg_order_tsvd = np.argsort(deg_recon_tsvd)
deg_seq_recon_tsvd = deg_recon_tsvd[deg_order_tsvd[::-skip_int]]

adj_recon_sort = np.copy(adj_recon_tsvd[np.ix_(deg_order_tsvd, deg_order_tsvd)])

pool = Pool(5)
tri_seq_recon_tsvd = np.array(pool.map(get_num_tri, np.arange(n)[::-skip_int]))
pool.close()
pool.join()

In [ ]:
deg_true = np.array(adj.sum(axis=1)).flatten()
deg_order_true = np.argsort(deg_true)
deg_seq_true = deg_recon[deg_order[::-skip_int]]

adj_recon_sort = np.copy(adj.todense()[np.ix_(deg_order_true, deg_order_true)])

pool = Pool(5)
tri_seq_true = np.array(pool.map(get_num_tri, np.arange(adj.shape[0])[::-skip_int]))
pool.close()
pool.join()

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(deg_seq_recon_lpca, tri_seq_recon_lpca / n, label='LPCA')
ax.semilogy(deg_seq_recon_tsvd, tri_seq_recon_tsvd / n, label='TSVD')
ax.semilogy(deg_seq_true, tri_seq_true / n, label='True', linestyle='--')

ax.legend()
ax.grid()
ax.autoscale(enable=True, axis='x', tight=True)
ax.autoscale(enable=True, axis='y', tight=True)
ax.set_xlim(1,)
ax.set_ylim(1/n,)

ax.set_ylabel('Triangles per Node')
ax.set_xlabel('Degree Upper Bound')

fig.set_dpi(300)